In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! cp /content/drive/My\ Drive/EDF_norm.zip /content/
! cp /content/drive/My\ Drive/output_frame_cells_count.csv /content/

# # !rm 'EDF_norm.zip'
# # !ls /content/drive/My\ Drive/

In [0]:
!unzip EDF_norm.zip

In [0]:
!ls /content/EDF_norm

In [0]:
from __future__ import absolute_import, division, print_function

import pathlib
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import numpy as np
from tensorflow import keras
import tensorflow as tf
#from tensorflow.keras import layers

print(tf.__version__)

1.13.1


In [0]:
path = "/content/EDF_norm/"
files = os.listdir(path)
files = sorted([file for file in files if ".png" in file])
X = np.zeros(shape=(len(files), 800, 800, 1), dtype=np.float16)
for index, file in enumerate(files):
  image = io.imread(os.path.join(path, file))
  image = image / np.max(image)
  X[index, :, :, 0] = image

# X = np.array([ np.array(io.imread(path + fname)) for fname in files])
print(X.shape)

(93, 800, 800, 1)


In [0]:
# X = np.reshape(X, (93, 800,800,1))
# X.shape

In [0]:
df = pd.read_csv('/content/output_frame_cells_count.csv')
Y = df['count'].values
Y.shape # type(Y)= numpy.ndarray

(93,)

In [0]:
# increase dimension by 1

#option to shuffle

In [0]:
x_train = X[:75] # 75 images
x_test = X[75:90] # 25 images

y_train = Y[:75]
y_test = Y[75:90]

In [0]:
def build_model():
    model = keras.models.Sequential([
    keras.layers.Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=(800,800, 1)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(), 
    keras.layers.Dense(500, activation='relu'),
    keras.layers.Dense(1)
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-04), loss='mean_squared_error', metrics=['accuracy'])
    return model

# model.compile(loss='mean_squared_error', optimizer=optimizer,
#  metrics=['mean_absolute_error', 'mean_squared_error'])


model = build_model()
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 798, 798, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 399, 399, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 399, 399, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 397, 397, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 198, 198, 32)      0         
_______________________________________

In [0]:
class TestCallback(keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

        
# model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
#           callbacks=[TestCallback((X_test, Y_test))])

In [0]:
EPOCHS = 3000

history = model.fit(
    x_train, y_train,
    epochs=EPOCHS, 
    validation_split = 0.2,
    verbose=2, 
    batch_size=4,
    callbacks=[TestCallback((x_test, y_test))], 
    
)

Train on 60 samples, validate on 15 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3000

Testing loss: 239.13375854492188, acc: 0.0

 - 116s - loss: 765.6091 - acc: 0.0000e+00 - val_loss: 1625.3924 - val_acc: 0.0000e+00
Epoch 2/3000

Testing loss: 251.15061950683594, acc: 0.0

 - 110s - loss: 437.6905 - acc: 0.0000e+00 - val_loss: 1645.0583 - val_acc: 0.0000e+00
Epoch 3/3000

Testing loss: 231.50466918945312, acc: 0.0

 - 111s - loss: 380.6628 - acc: 0.0000e+00 - val_loss: 1553.1007 - val_acc: 0.0000e+00
Epoch 4/3000

Testing loss: 198.3865203857422, acc: 0.0

 - 111s - loss: 319.3882 - acc: 0.0000e+00 - val_loss: 1397.8656 - val_acc: 0.0000e+00
Epoch 5/3000

Testing loss: 200.52163696289062, acc: 0.0

 - 110s - loss: 338.8399 - acc: 0.0000e+00 - val_loss: 1392.3913 - val_acc: 0.0000e+00
Epoch 6/3000

Testing loss: 209.8732147216797, acc: 0.0

 - 109s - loss: 287.0354 - acc: 0.0000e+00 - val_loss: 1415.9268 - val_acc: 0.0000e+00
Epoch 7/3000

Testing loss: 199.72201538

In [0]:
model.evaluate(x_test, y_test)

15/15 [==============================] - 3s 202ms/sample - loss: 3217372.0000 - acc: 0.0000e+00


[3217372.0, 0.0]

In [0]:
model.predict(x_test), y_test

(array([[6.564193 ],
        [6.9798036],
        [7.0459957],
        [5.866261 ],
        [7.43832  ],
        [7.5253315],
        [6.0166283],
        [5.6070266],
        [7.185611 ],
        [6.3236146],
        [6.9969583],
        [6.908541 ],
        [6.522974 ],
        [6.6133194],
        [7.0095625]], dtype=float32),
 array([58, 23,  8, 26, 16, 17, 12, 13,  8,  8,  5,  9, 13, 21, 12]))

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch￼
hist.tail()

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()


In [0]:
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
